be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [ ]:
import pandas as pd
import sqlite3

import scoped_mapping

from datetime import datetime

from functools import reduce

# User-provided data
See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

In [ ]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file = "../target/harmonized_table.db"

- soil
- sediment
- plant-associated

In [ ]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)

selected_package = "sediment"

In [ ]:
# hard to do with pure sql in sqlite3 (no full outer jons,
# just nested unions https://www.sqlitetutorial.net/sqlite-full-outer-join/)

# selected_package = "sediment"
# selected_package = "plant-associated"

print(selected_package)

repaireds = []
mixs_triad_parts = ["env_broad_scale", "env_local_scale", "env_medium"]


q = (
    """select
	rep.id,
	rep.env_package_orig ,
	rep.env_package_rep ,
	b.env_broad_scale ,
	b.env_local_scale ,
	b.env_medium
from
	repaired_env_package rep
join biosample b
		using(id)
where
	env_package_rep = '"""
    + selected_package
    + "'"
)
[repaired_env_package, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

repaired_env_package = repaired_env_package.rename(
    columns={"env_package_rep": "env_package"}
)

for current_part in mixs_triad_parts:
    print(current_part)
    repaired_env_package = repaired_env_package.rename(
        columns={current_part: current_part + "_orig"}
    )
    
repaireds.append(repaired_env_package)
# print(repaired_env_package)

some sediment samples don't appear in repaired_env_package?


BIOSAMPLE:SAMEA2531522 is getting picked up by sediment and plant-associated?

In [ ]:
print(selected_package)
'BIOSAMPLE:SAMEA2531522' in list(repaired_env_package['id'])

In [ ]:
for current_part in mixs_triad_parts:
    print(current_part)
    q = "select * from repaired_" + current_part
    [repaired, query_duration] = scoped_mapping.timed_query(
        q, biosample_cnx, print_timing=True
    )
    #     print(repaired.columns[1])
    repaired = repaired.drop("notes", axis=1)
    repaired = repaired.drop(repaired.columns[1], axis=1)
    #     repaired = repaired.rename(columns={repaired.columns[1]: current_part + "_orig"})
    repaired = repaired.rename(columns={"consensus_id": current_part})
    repaired = repaired.rename(columns={"consensus_lab": current_part + "_label"})
    repaireds.append(repaired)
#     print(repaired)


# will need to fish out the unique non-null value
# assume at least one ebs has been repaired and so can provide the notes value?
q = "select distinct notes from repaired_env_broad_scale"
[notes, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

In [ ]:
notes = list(notes["notes"])

notes = [i for i in notes if i]

notes = notes[0]

In [ ]:
nan_value = ""

repaired_mixs_plus = reduce(
    lambda left, right: pd.merge(left, right, on=["id"], how="outer"), repaireds
).fillna(nan_value)

repaired_mixs_plus["notes"] = notes

In [ ]:
repaired_mixs_plus
# lacks original values when no repair was made

In [ ]:
# mixs_plus_repaired.to_csv("repaired_soil_packages.tsv", sep="\t", index=False)

In [ ]:
repaired_mixs_plus.to_sql(
    "repaired_mixs_plus", biosample_cnx, if_exists="append", index=False
)

```sql
select
	env_package ,
	count(1)
from
	repaired_mixs_plus rmp
group by
	env_package 
order by
	count(1);
```